# Tutorial 9 - Changing the mesh

In [Tutorial 8](./Tutorial%207%20-%20Solver%20options.ipynb) we saw how to change the solver options. In this tutorial we will change the mesh used in the simulation, and show how to investigate the influence of the mesh on the solution.

All models in PyBaMM have a default number of mesh points used in a simulation. However, depending on things like the operating conditions you are simulating or the parameters you are using, you may find you need to increase the number points in the mesh to obtain an accurate solution. On the other hand, you may find that you are able to decrease the number of mesh points and still obtain a solution with an acceptable degree of accuracy but in a shorter amount of computational time. 

It is always good practice to conduct a mesh refinement study, where you simulate the same problem with a finer mesh and compare the results. Here will show how to do this graphically, but in practice you may wish to do a more detailed calculation of the relative error.

In [1]:
%pip install pybamm -q    # install PyBaMM if it is not installed
import pybamm

Note: you may need to restart the kernel to use updated packages.


## Changing the number of points in the mesh

First we load a model

In [2]:
model = pybamm.lithium_ion.SPMe()

We can then look at the default number of points, which are stored as a dictionary whose keys are the variables for each domain

In [3]:
model.default_var_pts

{SpatialVariable(0x62eb5d16885394c9, x_n, children=[], domain=['negative electrode'], auxiliary_domains={'secondary': "['current collector']"}): 20,
 SpatialVariable(-0x14459c29151bda45, x_s, children=[], domain=['separator'], auxiliary_domains={'secondary': "['current collector']"}): 20,
 SpatialVariable(-0x4a38fdaaecee173f, x_p, children=[], domain=['positive electrode'], auxiliary_domains={'secondary': "['current collector']"}): 20,
 SpatialVariable(-0xf387938c410c357, r_n, children=[], domain=['negative particle'], auxiliary_domains={'secondary': "['negative electrode']", 'tertiary': "['current collector']"}): 30,
 SpatialVariable(0x2f93da3e03d4dcbe, r_p, children=[], domain=['positive particle'], auxiliary_domains={'secondary': "['positive electrode']", 'tertiary': "['current collector']"}): 30,
 SpatialVariable(0x14b0200ccfc1f5cc, y, children=[], domain=['current collector'], auxiliary_domains={}): 10,
 SpatialVariable(-0x4938b510c47f7708, z, children=[], domain=['current collect

To run a simulation with a different number of points we can define our own dictionary 

In [4]:
# get the spatial variables used in pybamm
var = pybamm.standard_spatial_vars  

# create our dictionary 
var_pts = {
    var.x_n: 10,  # negative electrode
    var.x_s: 10,  # separator 
    var.x_p: 10,  # positive electrode
    var.r_n: 10,  # negative particle
    var.r_p: 10,  # positive particle
}

We then create and solve a simulation, passing the dictionary of points as a keyword argument

In [5]:
sim = pybamm.Simulation(model, var_pts=var_pts)
sim.solve([0, 3600])

and plot the solution in the usual way

In [6]:
sim.plot()

interactive(children=(FloatSlider(value=0.0, description='t', max=1.0, step=0.01), Output()), _dom_classes=('w…

## Conducting a mesh refinement study

In order to investigate the influence of the mesh on the solution we must solve the model multiple times, increasing the mesh resolution as we go. We first create a list of the number of points per domain we would like to use

In [7]:
npts = [4, 8, 16, 32, 64]

and now we can loop over the list, creating and solving simulations as we go. The solutions are stored in the list `solutions`

In [8]:
# choose model and parameters
model = pybamm.lithium_ion.DFN()
chemistry = pybamm.parameter_sets.Ecker2015
parameter_values = pybamm.ParameterValues(chemistry=chemistry)

# choose solver 
solver = pybamm.CasadiSolver(mode="fast")

# loop over number of mesh points
solutions = []
for N in npts:
    var_pts = {
        var.x_n: N,  # negative electrode
        var.x_s: N,  # separator 
        var.x_p: N,  # positive electrode
        var.r_n: N,  # negative particle
        var.r_p: N,  # positive particle
    }    
    sim = pybamm.Simulation(
        model, solver=solver, parameter_values=parameter_values, var_pts=var_pts
    )
    sim.solve([0, 3600])
    solutions.append(sim.solution)


We can now pass our list of solutions to the dynamic plot method, allowing use to see the influence of the mesh on the computed terminal voltage. We pass our list of points using the `labels` keyword so that the plots are labeled with the number of points used in the simulation

In [9]:
pybamm.dynamic_plot(solutions, ["Terminal voltage [V]"], time_unit="seconds", labels=npts)    

interactive(children=(FloatSlider(value=0.0, description='t', max=3600.0, step=36.0), Output()), _dom_classes=…